In [4]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, r2_score


import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, Dropout, Concatenate

# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

In [22]:
# Determine treatment
assistments_usage = pd.read_csv('assistments_data/csv/school_year_data.csv', dtype=str).astype(str)
assistments_usage = assistments_usage.set_index('school_id')
assistments_usage = assistments_usage.astype(float)

mass_doe_data = pd.read_csv('mcas_exports/csv/mass_doe_data.csv', dtype=str).astype(str)
mass_doe_data = mass_doe_data.set_index('school_code')
float_columns = [c for c in mass_doe_data if c != 'prior_performance']
mass_doe_data[float_columns] = mass_doe_data[float_columns].astype(float)
mass_doe_data['prior_performance'] = mass_doe_data['prior_performance'].apply(lambda x: np.array(eval(x.replace('nan', 'np.nan'))))

In [74]:
treatment = assistments_usage[assistments_usage['school_end_year'].isin([2012, 2013])]
treatment = treatment.groupby('school_id')['assignment_log_count'].mean()
treatment = treatment.rename('treatment_usage')

pretreatment = assistments_usage[assistments_usage['school_end_year'] < 2012]
pretreatment = pretreatment.groupby('school_id')['assignment_log_count'].mean()
pretreatment = pretreatment.rename('pretreatment_usage')

population = mass_doe_data['6'] + mass_doe_data['7'] + mass_doe_data['8']
population = population.rename('population')

participation = pd.concat([treatment, pretreatment, population], axis=1, sort=True)
participation = participation.dropna(subset=['treatment_usage', 'population'])
participation = participation.fillna(0)
treatment_check = (participation['treatment_usage'] / participation['population'] >= 1)
pretreatment_check = (participation['pretreatment_usage'] / participation['population'] < 0.5)
participants = participation.index[treatment_check & pretreatment_check]

mass_doe_data['in_treatment'] = mass_doe_data.index.isin(participants).astype(int)
mass_doe_data['in_remnant'] = 1 - mass_doe_data.index.isin(assistments_usage.index).astype(int)

In [ ]:
# Get model stats and propensity scores for every school

propensity_scores = []
aucs = []
r2s = []
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True, random_state=0).split(ff_X, y):
    
    # Clear session so models don't pile up
    keras.backend.clear_session()
    
    # Split data into training and testing splits
    train_ff_X, test_ff_X = ff_X[train_index], ff_X[test_index]
    train_lstm_X, test_lstm_X = lstm_X[train_index], lstm_X[test_index]
    train_y, test_y = y[train_index], y[test_index]
    train_w, test_w = w[train_index], w[test_index]
    
    # Normalize the input data based on the training data distribution
    ff_scaler = StandardScaler().fit(train_ff_X)
    train_ff_X = np.nan_to_num(ff_scaler.transform(train_ff_X))
    test_ff_X = np.nan_to_num(ff_scaler.transform(test_ff_X))
    
    train_lstm_X_shape = train_lstm_X.shape
    train_stacked_lstm_X = train_lstm_X.reshape(-1, train_lstm_X_shape[-1])
    lstm_scaler = StandardScaler().fit(train_stacked_lstm_X)
    train_lstm_X = np.nan_to_num(lstm_scaler.transform(train_stacked_lstm_X)).reshape(train_lstm_X_shape)
    test_lstm_X_shape = test_lstm_X.shape
    test_stacked_lstm_X = test_lstm_X.reshape(-1, test_lstm_X_shape[-1])
    test_lstm_X = np.nan_to_num(lstm_scaler.transform(test_stacked_lstm_X)).reshape(test_lstm_X_shape)
    
    # Create the neural network
    ff_input_layer = Input(shape=train_ff_X[0].shape)
    ff_model = Dense(units=64, activation='tanh')(ff_input_layer)
    ff_model = Dropout(rate=0.5)(ff_model)
    
    lstm_input_layer = Input(shape=train_lstm_X[0].shape)
    lstm_model = Masking(mask_value=0.0)(lstm_input_layer)
    lstm_model = LSTM(units=64, return_sequences=False, activation='tanh', dropout=0.5, recurrent_dropout=0.5)(lstm_model)
    
    #model = Concatenate()([ff_model, lstm_model])
    model = ff_model
    output_layer = Dense(units=1, activation='sigmoid')(model)
    
    #combined_model = Model([ff_input_layer, lstm_input_layer], output_layer)
    combined_model = Model(ff_input_layer, output_layer)
    combined_model.compile(optimizer='adam', loss='binary_crossentropy')
    
    # Train the neural network
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    combined_model.fit(x=train_ff_X, #x=[train_ff_X, train_lstm_X],
                       y=train_y,
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=train_w,
                       verbose=0)
    
    # Use the neural network to predict the held-out fold
    #pred_y = combined_model.predict([test_ff_X, test_lstm_X]).flatten()
    pred_y = combined_model.predict(test_ff_X).flatten()
    
    # Update propensity scores and metrics
    propensity_scores.extend(pred_y.tolist())
    aucs.append(roc_auc_score(test_y.flatten(), pred_y))
    r2s.append(r2_score(test_y.flatten(), pred_y))

In [ ]:
# Display Findings

print(f'Model AUC = {np.mean(aucs):.4g}')
print(f'Model R\u00b2 = {np.mean(r2s):.4g}')

plt.figure()
plt.hist(propensity_scores, bins=20)
plt.xlim([0,1])
plt.xlabel('Propensity Score')
plt.ylabel('Prediction Frequency')
plt.title('Frequency of Propensity Scores')
plt.show()

In [ ]:
Model AUC = 0.5519
Model R² = -10.7